# Exercise 3: Durable Agent — Solution

Complete implementation combining OpenAI agents with Temporal for durability.

## Prerequisites

Ensure the Temporal dev server is running and `.env` contains your OpenAI credentials.

In [ ]:
%pip install --quiet temporalio openai rich

import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.common import RetryPolicy

console = Console()


## Run the Solution

Execute the workflow and observe durable behavior with retries.

In [ ]:
# Run the durable agent solution
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 3: Durable Agent — Solution[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "durable-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[DurableAgentWorkflow],
        activities=[call_agent_with_tools],
    ):
        query = "What's the weather like in San Francisco?"
        workflow_id = f"durable-agent-{trace_id}"

        console.print(f"[yellow]Query:[/yellow] {query}\n")

        result = await client.execute_workflow(
            DurableAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]🤖 Agent Response:[/bold green]\n{result}\n")
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[yellow]Trace ID for correlation:[/yellow] {trace_id}\n")

asyncio.run(run_solution())


## Source Code

Inspect the solution modules below.

In [ ]:
def get_weather(location: str) -> str:
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    weather = weather_data.get(location, "partly cloudy, 68°F")
    return f"The weather in {location} is {weather}"


@activity.defn
async def call_agent_with_tools(query: str, trace_id: str) -> str:
    activity.logger.info("🤖 Activity started")
    activity.logger.info(f"   Query: {query}")
    activity.logger.info(f"   Trace ID: {trace_id}")

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    messages = [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        activity.logger.info(f"🔧 Tool calls detected: {len(tool_calls)}")
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = eval(tool_call.function.arguments)
            activity.logger.info(f"   Calling: {function_name}({function_args})")

            if function_name == "get_weather":
                function_response = get_weather(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        final_message = final_response.choices[0].message.content
    else:
        final_message = response_message.content

    activity.logger.info("✅ Activity completed")
    return final_message


@workflow.defn
class DurableAgentWorkflow:
    @workflow.run
    async def run(self, query: str, trace_id: str) -> str:
        workflow.logger.info("🚀 Durable agent workflow started")
        workflow.logger.info(f"   Query: {query}")
        workflow.logger.info(f"   Trace ID: {trace_id}")

        result = await workflow.execute_activity(
            call_agent_with_tools,
            args=[query, trace_id],
            start_to_close_timeout=timedelta(seconds=30),
            retry_policy=RetryPolicy(
                initial_interval=timedelta(seconds=1),
                maximum_interval=timedelta(seconds=10),
                maximum_attempts=3,
                backoff_coefficient=2.0,
            ),
        )

        workflow.logger.info("✅ Workflow completed successfully")
        return result
